In [58]:
import os
os.environ['CUDA_VISIBLE_DEVICES']="0"
import pandas as pd
import numpy as np
from collections import defaultdict
from itertools import combinations
from itertools import chain
from collections import namedtuple
import pickle
import tensorflow as tf
import tflearn
import matplotlib.pyplot as plt
import os.path
import shutil
import inspect
import matplotlib.pyplot as plt
import json
from pprint import pprint
from copy import copy
from tflearn.datasets import mnist
from nn_lib import *
from train_utils import *
from train_procedures import *
from mnist_utils import *
import sys
np.random.seed(1997)
ALGORITHM=sys.argv[1]
BASE_DIR=sys.argv[2]
if ALGORITHM=='full_data':
    CLASS_BATCH_SIZE=int(sys.argv[3])
elif ALGORITHM=="SupportNet_exemplars_set_size":
    EXEMPLARS_SET_SIZE=int(sys.argv[3])

In [3]:
X_train,Y_train,X_test,Y_test=data_dict=mnist.load_data()
data_dict={'X_train':X_train,'Y_train':Y_train,'X_test':X_test,'Y_test':Y_test}

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


In [4]:
data_dict=data_dict_resize(data_dict)

In [6]:
if ALGORITHM=='ft':
#non-incremental settings
    hyper_params={'beta':1e-5,
                  'initial_lr':0.1,
                  'train_batch_size':64,
                  'test_batch_size':64,
                  'lr_reduction_rate':10,
                  'lr_reduction_epoch':[10,20],
                  'use_fixedsize_exemplars':True,
                  'exemplars_set_size':2000,
                  'final_train_epochs':0,
                  'se':True,
                  'primary_exemplars':None,
                  'train_method':'train_plain',
                  'sample_weight':0,
                  'shuffle_class_ord':False,
                  'optimizer':'momentum',
                  'ewc_lambda':0,
                  'loss_function':'softmax_cross_entropy_with_logits'}
    fixed_params={'dataset':'mnist',
                  'net_type':'ResNet32',
                  'random_seed':1997,
                  'total_num_classes':10,
                  'base_dir':BASE_DIR,
                  'class_batch_size':2,
                  'use_theoretical_mean':True}
elif ALGORITHM=='icarl':
  #icarl settings
    hyper_params={'beta':1e-3,
                  'initial_lr':0.1,
                  'train_batch_size':64,
                  'test_batch_size':64,
                  'lr_reduction_rate':10,
                  'lr_reduction_epoch':[10,20],
                  'use_fixedsize_exemplars':True,
                  'exemplars_set_size':2000,
                  'final_train_epochs':5,
                  'se':True,
                  'primary_exemplars':'icarl_exemplars',
                  'train_method':'train_distillation',
                  'sample_weight':0,
                  'shuffle_class_ord':False,
                  'optimizer':'momentum',
                  'ewc_lambda':1e-2,
                  'loss_function':'sigmoid_cross_entropy_with_logits'}
    fixed_params={'dataset':'mnist',
                  'net_type':'ResNet32',
                  'random_seed':1997,
                  'total_num_classes':10,
                  'base_dir':BASE_DIR,
                  'class_batch_size':2,
                  'use_theoretical_mean':True} 
elif ALGORITHM=='SupportNet':
      #SupportNet settings
    hyper_params={'beta':1e-5,
                  'initial_lr':0.1,
                  'train_batch_size':64,
                  'test_batch_size':64,
                  'lr_reduction_rate':10,
                  'lr_reduction_epoch':[10,20],
                  'use_fixedsize_exemplars':True,
                  'exemplars_set_size':2000,
                  'final_train_epochs':5,
                  'se':True,
                  'primary_exemplars':'svm_exemplars',
                  'train_method':'train_plain',
                  'sample_weight':0,
                  'shuffle_class_ord':False,
                  'optimizer':'momentum',
                  'ewc_lambda':1e-2,
                  'loss_function':'sigmoid_cross_entropy_with_logits'}
    fixed_params={'dataset':'mnist',
                  'net_type':'ResNet32',
                  'random_seed':1997,
                  'total_num_classes':10,
                  'base_dir':BASE_DIR,
                  'class_batch_size':2,
                  'use_theoretical_mean':True}  
elif ALGORITHM=="SupportNet_noewc":
    hyper_params={'beta':1e-5,
                  'initial_lr':0.1,
                  'train_batch_size':64,
                  'test_batch_size':64,
                  'lr_reduction_rate':10,
                  'lr_reduction_epoch':[10,20],
                  'use_fixedsize_exemplars':True,
                  'exemplars_set_size':2000,
                  'final_train_epochs':5,
                  'se':True,
                  'primary_exemplars':'svm_exemplars',
                  'train_method':'train_plain',
                  'sample_weight':0,
                  'shuffle_class_ord':False,
                  'optimizer':'momentum',
                  'ewc_lambda':0,
                  'loss_function':'sigmoid_cross_entropy_with_logits'}
    fixed_params={'dataset':'mnist',
                  'net_type':'ResNet32',
                  'random_seed':1997,
                  'total_num_classes':10,
                  'base_dir':BASE_DIR,
                  'class_batch_size':2,
                  'use_theoretical_mean':True}  
elif ALGORITHM=="SupportNet_exemplars_set_size":
    hyper_params={'beta':1e-5,
                  'initial_lr':0.1,
                  'train_batch_size':64,
                  'test_batch_size':64,
                  'lr_reduction_rate':10,
                  'lr_reduction_epoch':[10,20],
                  'use_fixedsize_exemplars':True,
                  'exemplars_set_size':EXEMPLARS_SET_SIZE,
                  'final_train_epochs':5,
                  'se':True,
                  'primary_exemplars':'svm_exemplars',
                  'train_method':'train_plain',
                  'sample_weight':0,
                  'shuffle_class_ord':False,
                  'optimizer':'momentum',
                  'ewc_lambda':1e-2,
                  'loss_function':'sigmoid_cross_entropy_with_logits'}
    fixed_params={'dataset':'mnist',
                  'net_type':'ResNet32',
                  'random_seed':1997,
                  'total_num_classes':10,
                  'base_dir':BASE_DIR,
                  'class_batch_size':2,
                  'use_theoretical_mean':True} 
elif ALGORITHM == 'ewc':
    #ewc settings
    hyper_params={'beta':2e-5,
                  'initial_lr':0.1,
                  'train_batch_size':64,
                  'test_batch_size':64,
                  'lr_reduction_rate':10,
                  'lr_reduction_epoch':[10,20],
                  'use_fixedsize_exemplars':True,
                  'exemplars_set_size':2000,
                  'final_train_epochs':0,
                  'se':True,
                  'primary_exemplars':None,
                  'train_method':'train_plain',
                  'sample_weight':0,
                  'shuffle_class_ord':False,
                  'optimizer':'momentum',
                  'ewc_lambda':1e-2,
                  'loss_function':'sigmoid_cross_entropy_with_logits'}
    fixed_params={'dataset':'mnist',
                  'net_type':'ResNet32',
                  'random_seed':1997,
                  'total_num_classes':10,
                  'base_dir':BASE_DIR,
                  'class_batch_size':2,
                  'use_theoretical_mean':True}  
elif ALGORITHM == 'full_data':
    hyper_params={'beta':1e-5,
                  'initial_lr':0.1,
                  'train_batch_size':64,
                  'test_batch_size':64,
                  'lr_reduction_rate':10,
                  'lr_reduction_epoch':[10,20],
                  'use_fixedsize_exemplars':True,
                  'exemplars_set_size':2000,
                  'final_train_epochs':0,
                  'se':True,
                  'primary_exemplars':None,
                  'train_method':'train_plain',
                  'sample_weight':0,
                  'shuffle_class_ord':False,
                  'optimizer':'momentum',
                  'ewc_lambda':0,
                  'loss_function':'sigmoid_cross_entropy_with_logits'}
    fixed_params={'dataset':'mnist',
                  'net_type':'ResNet32',
                  'random_seed':1997,
                  'total_num_classes':10,
                  'base_dir':BASE_DIR,
                  'class_batch_size':CLASS_BATCH_SIZE,
                  'use_theoretical_mean':True}

else:
    assert False

In [7]:
tf_tensors,tf_variables,tf_networks=build_graph(hyper_params,fixed_params)

Use ResNet32


/home/liz0f/SupportNet/train_procedures.py:50: UserWarning: when use ewc, loss_function should be softmax
  warnings.warn('when use ewc, loss_function should be softmax')


Use ResNet32
Use ResNet32


In [8]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
sess=tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [103]:
if ALGORITHM=='full_data':
    fit(tf_tensors,tf_variables,tf_networks,fixed_params,hyper_params,data_dict,sess,resume=False,
            save_params=True,evaluation_metric='top1_accuracy',save_history=True,
            num_epochs=30,num_iterations=1,verbose=2,print_freq=1,pretrain_evaluation=1,override_warning=False)
else:
    fit(tf_tensors,tf_variables,tf_networks,fixed_params,hyper_params,data_dict,sess,resume=False,
            save_params=True,evaluation_metric='top1_accuracy',save_history=True,
            num_epochs=30,num_iterations=5,verbose=2,print_freq=1,pretrain_evaluation=1,override_warning=False)

{'base_dir': 'Sep16-time_test2',
 'class_batch_size': 2,
 'dataset': 'mnist',
 'net_type': 'ResNet32',
 'random_seed': 1997,
 'total_num_classes': 10,
 'use_theoretical_mean': True}
{'beta': 1e-05,
 'ewc_lambda': 0.01,
 'exemplars_set_size': 2000,
 'final_train_epochs': 5,
 'initial_lr': 0.1,
 'loss_function': 'sigmoid_cross_entropy_with_logits',
 'lr_reduction_epoch': [10, 20],
 'lr_reduction_rate': 10,
 'optimizer': 'momentum',
 'primary_exemplars': 'svm_exemplars',
 'sample_weight': 0,
 'se': True,
 'shuffle_class_ord': False,
 'test_batch_size': 64,
 'train_batch_size': 64,
 'train_method': 'train_plain',
 'use_fixedsize_exemplars': True}
Sep16-time_test2 already exists, override?y
Y_train class info:
Counter({1: 6179, 0: 5444})
Y_test class info:
Counter({1: 1135, 0: 980})
===========Iteration 1=============
Using classes [0, 1]


/home/liz0f/.conda/envs/tf/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/liz0f/.conda/envs/tf/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/liz0f/.conda/envs/tf/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/liz0f/.conda/envs/tf/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/liz0

Class batch pretrain evaluation (plain method)
	Train loss: 3.187233
	Train class loss: 3.172697
	Train reg loss: 0.014536
	Validation loss: 3.204652
	Validation class loss: 3.190116
	Validation reg loss: 0.014536
	Top1 train accuracy: 0.000000
	Top5 train accuracy: 0.468382
	Top1 validation accuracy: 0.000000
	Top5 validation accuracy: 0.463357
[[  0   0   0 980]
 [  0   0 585 550]
 [  0   0   0   0]
 [  0   0   0   0]]
Epoch 1
	Train loss: 0.035624
	Train class loss: 0.021061
	Train reg loss: 0.014563
	Validation loss: 0.015528
	Validation class loss: 0.000925
	Validation reg loss: 0.014603
	Top1 train accuracy: 0.975136
	Top5 train accuracy: 0.990622
	Top1 validation accuracy: 0.990071
	Top5 validation accuracy: 1.000000
[[ 959   21]
 [   0 1135]]
mbi=291175168
time= 34.478517
saving model parameters...
Epoch 2
	Train loss: 0.015784
	Train class loss: 0.001252
	Train reg loss: 0.014533
	Validation loss: 0.014947
	Validation class loss: 0.000387
	Validation reg loss: 0.014560
	Top1 t

Epoch 2
	Train loss: 0.021408
	Train class loss: 0.006417
	Train reg loss: 0.014459
	Validation loss: 0.015220
	Validation class loss: 0.000163
	Validation reg loss: 0.014506
	Top1 train accuracy: 0.987107
	Top5 train accuracy: 0.996033
	Top1 validation accuracy: 0.998041
	Top5 validation accuracy: 1.000000
[[1031    1]
 [   3 1007]]
mbi=291543552
time= 39.208088
saving last-epoch parameters...
On iteration 2
Plain evaluation before retrain
	Best top1 validation accuracy: 0.998041
	Best top5 validation accuracy: 1.000000
	Best top1 cumul accuracy: 0.846283
	Best top5 cumul accuracy: 0.994227
	Best top1 ori accuracy: 0.699764
	Best top5 ori accuracy: 0.988652
	Report string of cumul dataset
             precision    recall  f1-score   support

          0       0.69      0.98      0.81       980
          1       1.00      0.46      0.63      1135
          2       0.92      1.00      0.96      1032
          3       0.89      1.00      0.94      1010

avg / total       0.88      0.85  

Epoch 2
	Train loss: 0.024062
	Train class loss: 0.009475
	Train reg loss: 0.014504
	Validation loss: 0.015509
	Validation class loss: 0.000911
	Validation reg loss: 0.014506
	Top1 train accuracy: 0.986335
	Top5 train accuracy: 0.998943
	Top1 validation accuracy: 0.988260
	Top5 validation accuracy: 1.000000
[[  0   0   0   0   0   0]
 [  0   0   0   0   0   0]
 [  0   0   0   0   0   0]
 [  0   0   0   0   0   0]
 [  0   6   6   0 970   0]
 [  1   0   0   9   0 882]]
mbi=291790080
time= 36.788856
saving last-epoch parameters...
On iteration 3
Plain evaluation before retrain
	Best top1 validation accuracy: 0.988260
	Best top5 validation accuracy: 1.000000
	Best top1 cumul accuracy: 0.989222
	Best top5 cumul accuracy: 1.000000
	Best top1 ori accuracy: 0.993381
	Best top5 ori accuracy: 1.000000
	Report string of cumul dataset
             precision    recall  f1-score   support

          0       1.00      0.99      0.99       980
          1       0.99      1.00      0.99      1135
     

===========Iteration 4=============
Using classes [6, 7]
Class batch pretrain evaluation (plain method)
	Train loss: 0.087735
	Train class loss: 0.073229
	Train reg loss: 0.014506
	Validation loss: 0.100641
	Validation class loss: 0.086135
	Validation reg loss: 0.014506
	Top1 train accuracy: 0.134196
	Top5 train accuracy: 0.163823
	Top1 validation accuracy: 0.000000
	Top5 validation accuracy: 0.018127
[[  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [136  52 458   6  99 207   0   0]
 [  1 220  12 735  41  19   0   0]]
Epoch 1
	Train loss: 0.077529
	Train class loss: 0.062707
	Train reg loss: 0.014488
	Validation loss: 0.015906
	Validation class loss: 0.000990
	Validation reg loss: 0.014517
	Top1 train accuracy: 0.906245
	Top5 train accuracy: 0.984387
	Top1 validation accuracy: 0.990937
	Top5 validation accuracy: 1.000000
[[   

Y_train class info:
Counter({9: 5454, 8: 5389, 0: 250, 1: 250, 2: 250, 3: 250, 4: 250, 5: 250, 6: 250, 7: 250})
Y_test class info:
Counter({9: 1009, 8: 974})
===========Iteration 5=============
Using classes [8, 9]
Class batch pretrain evaluation (plain method)
	Train loss: 0.093780
	Train class loss: 0.079266
	Train reg loss: 0.014514
	Validation loss: 0.106781
	Validation class loss: 0.092267
	Validation reg loss: 0.014514
	Top1 train accuracy: 0.106128
	Top5 train accuracy: 0.159620
	Top1 validation accuracy: 0.000000
	Top5 validation accuracy: 0.004034
[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   1 501  42   2  93 334   1   0   0]
 [  5   0   6   3  69 232   1 693   0   0]]
Epoch 

In [ ]:
data_dict_iterations=data_dict_each_iteration("Aug31-mnist-icarl-full2",5,2,data_dict)

In [ ]:
eval_vals_lists_test=eval_vals_each_iteration("Aug31-mnist-icarl-full2",5,2,hyper_params["test_batch_size"],tf_tensors,tf_networks,data_dict_iterations,"mnist",sess)

In [ ]:
for i in range(0,len(eval_vals_lists_test)):
    for j in range(0,len(eval_vals_lists_test)):
        print(eval_vals_lists_test[i][j]["top1_accuracy"],end=",")
    print()

In [ ]:
data_dict_first_iteration=data_dict_each_iteration("Aug31-mnist-SupportNet-full2",1,2,data_dict)

In [82]:
feature_list=feature_each_iteration("Nov15-mnist-SupportNet_noewc-full2",5,hyper_params["test_batch_size"],tf_tensors,tf_networks,data_dict_first_iteration[0]["X_train"],"mnist",sess)

In [83]:
with open(os.path.join("Nov15-mnist-SupportNet_noewc-full2","first_iteration_features.pkl"),"wb") as f:
    feature_dict=dict(feature_list=feature_list,labels=data_dict_first_iteration[0]["Y_train"])
    pickle.dump(feature_dict,f)

In [ ]:
table2_supportnet=table2("Aug31-mnist-SupportNet-full2",0,4,2,hyper_params["test_batch_size"],tf_tensors,tf_networks,data_dict,data_dict_iterations,"mnist",sess)

In [ ]:
for k,v in table2_supportnet.items():
    print("%s,%f"%(k,v["top1_accuracy"]))

In [ ]:
table2_icarl=table2("Aug31-mnist-icarl-full2",0,4,2,hyper_params["test_batch_size"],tf_tensors,tf_networks,data_dict,data_dict_iterations,"mnist",sess)

In [ ]:
for k,v in table2_icarl.items():
    print("%s,%f"%(k,v["top1_accuracy"]))

In [85]:
from print_result_summary import print_algorithm_result

In [89]:
algorithms_folders=["Nov15-mnist-SupportNet_exemplars_set_size-200exemplars-full2",
                    "Nov15-mnist-SupportNet_exemplars_set_size-500exemplars-full2",
                    "Nov15-mnist-SupportNet_exemplars_set_size-1000exemplars-full2",
                    "Nov15-mnist-SupportNet_exemplars_set_size-1500exemplars-full2"]

In [90]:
entries=['best_plain_before_cumul','best_plain_before_cumul','best_plain_before_cumul','best_plain_before_cumul']
metrics=['top1_accuracy','ck_score','f1_macro']
iterations=[5]*4

In [91]:
print_algorithm_result(algorithms_folders,entries,metrics,iterations)

top1_accuracy
Nov15-mnist-SupportNet_exemplars_set_size-200exemplars-full2,1.0,0.9901371181140245,0.8501077764881446,0.9352625670450293,0.8444
Nov15-mnist-SupportNet_exemplars_set_size-500exemplars-full2,1.0,0.9944671638200625,0.9872326314044105,0.9695646750654858,0.9425
Nov15-mnist-SupportNet_exemplars_set_size-1000exemplars-full2,1.0,0.9966321866730815,0.9905488310396285,0.986653361606586,0.9743
Nov15-mnist-SupportNet_exemplars_set_size-1500exemplars-full2,0.9995271867612293,0.9968727447678615,0.9935334107113248,0.9901459398777598,0.985

ck_score
Nov15-mnist-SupportNet_exemplars_set_size-200exemplars-full2,1.0,0.986836781325,0.820446135379,0.925965966383,0.827036435053
Nov15-mnist-SupportNet_exemplars_set_size-500exemplars-full2,1.0,0.992615892817,0.984666732637,0.965197465599,0.936082065381
Nov15-mnist-SupportNet_exemplars_set_size-1000exemplars-full2,1.0,0.995505142425,0.988648863153,0.984737189042,0.971433549518
Nov15-mnist-SupportNet_exemplars_set_size-1500exemplars-full2,0.99904